# Reviewer/Contributor Heatmap Discussion Notebook


This notebook will be the development ground for an activity heatmap from the perspectives of: 
- Contributors (Someone who has had a PR merged on the specific segment of the codebase) 
- Reviewers (Someone who has reviewed a PR on the specific segment of the codebase)

The x axis will be a break down of the repository folders and source files. The first iteration will statically show the source directory folders and files. The next iteration will allow users to select a more granular view on specific sections of the codebase. A part of this notebook review will be to get suggestions and feedback on how to impliment this. 

The initial concept of the visualization is as followed:

User selects which specific repository in the repositories in their search bar to show on heat map
User specifies a time interval in days that a reviewer needs to show some form of activity in the overall repo set
Then one of the following plotly heat maps is used:
https://plotly.com/python/heatmaps/
https://plotly.com/python/2D-Histogram/

- x axis: date by month in descending order
- y axis: repository folder (or file, need to workshop this a bit)
- z axis (color): Number of reviewers that have been active in the time interval (relative to the month block)

This notebook does all of the preprocessing down to the visualization for we can try different strategies before putting it into 8Knot

In [ ]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import json
import plotly.express as px
import datetime as dt
import plotly
import math
from IPython.display import Image

# connect to db
import json
import os

paths = ["../../comm_cage.json", "comm_cage.json", "../../config.json", "../config.json", "config.json", "../../copy_cage-padres.json"]

for path in paths:
    if os.path.exists(path):
        with open(path) as config_file:
            config = json.load(config_file)
        break
else:
    raise FileNotFoundError(f"None of the config files found: {paths}")
        
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})


In [ ]:
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

Get repo_ids for augur data access 

In [ ]:
repo_urls = ['https://github.com/opendatahub-io/data-science-pipelines-operator','https://github.com/pulp/pulp-infra-ansible'
             ,'https://github.com/opendatahub-io/data-science-pipelines','https://github.com/chaoss/augur']

In [ ]:
from sqlalchemy import text, bindparam

repo_urls = [
    'https://github.com/opendatahub-io/data-science-pipelines-operator',
    'https://github.com/pulp/pulp-infra-ansible',
    'https://github.com/opendatahub-io/data-science-pipelines',
    'https://github.com/chaoss/augur'
]

stmt = text("""
    SELECT DISTINCT
        r.repo_id,
        r.repo_name
    FROM repo r
    JOIN repo_groups rg ON r.repo_group_id = rg.repo_group_id
    WHERE r.repo_git IN :urls
""").bindparams(bindparam("urls", expanding=True))

with engine.connect() as conn:
    rows = conn.execute(stmt, {"urls": repo_urls}).all()

repo_ids   = [row[0] for row in rows]
repo_names = [row[1] for row in rows]
print(repo_ids)
print(repo_names)

This visualization is to be done on a single repo but we will use the 4 repo_ids to test out the stategy on difference file structures and contributor density

In [ ]:
repo_id = 1

## File query and preprocessing

Query to get all the files identified in the repo_labor table for the specific repository. For each collection of repo_labor, every file in current (as of that collection) existance is a row. During the preprocessing steps we will only keep the entries from the most recent collection

In [ ]:
file_query = salc.sql.text(f"""
                SELECT 
                    rl.repo_id,
                    r.repo_name,
                    r.repo_path,
                    rl.rl_analysis_date,
                    rl.file_path,
                    rl.file_name
                FROM 
                    repo_labor rl,
                    repo r
                WHERE 
                    rl.repo_id = {repo_id} AND 
                    rl.repo_id = r.repo_id
                """)
df_file = pd.read_sql(file_query, con=engine)

df_file = df_file.reset_index()
df_file.drop("index", axis=1, inplace=True)

In [ ]:
df_file

The file path directly from the query has a lot of excess text that is not useful to us. The following 3 cells will be to slice the excess characters to only leave the file path

In [ ]:
# strings to hold the values for each column (always the same for every row of this query)
repo_name = df_file["repo_name"].iloc[0]
repo_path = df_file["repo_path"].iloc[0]
repo_id = str(df_file["repo_id"].iloc[0])

In [ ]:
# pattern found in each file path, used to slice to get only the root file path 
path_slice = repo_id + '-' + repo_path +  '/' + repo_name + '/'

In [ ]:
df_file["file_path"] = df_file["file_path"].str.rsplit(path_slice,n= 1).str[1]

Drop all of the columns not in the most recent collection 

In [ ]:
df_file = df_file[df_file["rl_analysis_date"] == df_file["rl_analysis_date"].max()]

In [ ]:
# drop unneccessary columns not needed after preprocessing steps
df_file = df_file.reset_index()
df_file.drop(["index","repo_id","repo_name","repo_path","rl_analysis_date" ], axis=1, inplace=True)

Results in the most current set of files and their path

In [ ]:
df_file

## Contributor information preprocessing

### Contributors per file

Query to get all of the contributors that have opened a pr that included each file 

In [ ]:
cntrb_query = salc.sql.text(f"""
                SELECT 
                    prf.pr_file_path as file_path, 
                    string_agg(DISTINCT CAST(pr.pr_augur_contributor_id AS varchar(20)), ',') AS cntrb_ids
                FROM
                    pull_requests pr, 
                    pull_request_files prf
                WHERE 
                    pr.pull_request_id = prf.pull_request_id AND 
                    pr.repo_id = {repo_id}
                GROUP BY prf.pr_file_path 
                """)
df_cntrb = pd.read_sql(cntrb_query, con=engine)

df_cntrb["cntrb_ids"] = df_cntrb["cntrb_ids"].str.split(",")
df_cntrb = df_cntrb.reset_index()
df_cntrb.drop("index", axis=1, inplace=True)

In [ ]:
df_cntrb

Query to get all of the contributors that have reviewed a pr that included each file - *will fill in when augur bug fix is in*

#### Combine df_files and contributor information 

Left join on df_files to only get the files that are currently in the repository and the contributors that have ever opened a pr that included edits on the file 

In [ ]:
df_file = pd.merge(df_file,df_cntrb, on = 'file_path', how = 'left' )
# replace nan with empty string to avoid errors in list comprehension
df_file.cntrb_ids.fillna('', inplace=True)

In [ ]:
df_file

## File preprocessing - group by selected directory 


Split file path by directory

In [ ]:
df_file = df_file.join(df_file['file_path'].str.split('/', expand=True))

In [ ]:
df_file

No real known explaination for this besides: if do, everything works, if not it doesnt

In [ ]:
# for some reason this fixes formating and turns into list, should find better way 
df_file["cntrb_ids"] = df_file.apply(
        lambda row: [x for x in row.cntrb_ids],
        axis=1,)

Get all of the folders/directories in the repository

In [ ]:
# take all of the files, split on the last instance of a / to get directories and top level files 
directories = df_file["file_path"].str.rsplit("/",n= 1).str[0].tolist()
directories = list(set(directories))

In [ ]:
# get all of the file names to filter out of the directory set
top_level_files = df_file["file_name"][df_file[1].isnull()]
directories = [f for f in directories if f not in top_level_files]

User inputted option, with handling corner case of top level directory. In 8knot this would be a searchable drop down of the directories

In [ ]:
#directory_choosen = "augur/application" 
directory_choosen = "Top Level" 
levels = directory_choosen.count('/')

In [ ]:
# format for top level directory 
if directory_choosen == "Top Level":
    directory_choosen = ""
    levels = -1

Get all of the files in the directory or nested in folders in the directory 

In [ ]:
df_dynamic_directory = df_file[df_file['file_path'].str.startswith(directory_choosen)]

Groupby the level above the selected directory for all files nested in folders are together. For each, create a list of all of the contributors who have contributed.

In [ ]:
df_dynamic_directory = df_dynamic_directory.groupby(levels+1)["cntrb_ids"].sum().reset_index().rename(columns={levels+1: "directory_value"})

In [ ]:
df_dynamic_directory

Set of cntrb_ids to confirm there are no duplicate cntrb_ids

In [ ]:
df_dynamic_directory["cntrb_ids"] = df_dynamic_directory.apply(
        lambda row: set(row.cntrb_ids),
        axis=1,)

### Contributors last activity in repository

Query for contributions with related contributor information. This query gets the following contributor actions: 
- Commits 
- Issues: open, close, comment 
- Pull Requests: open, close, merge, review, comment


In [ ]:
action_query = salc.sql.text(f"""
                SELECT
                        repo_id as id,
                        repo_name,
                        cntrb_id,
                        created_at,
                        login,
                        action,
                        rank
                    FROM
                        augur_data.explorer_contributor_actions
                    WHERE
                        repo_id ={repo_id}
                """)
df_actions = pd.read_sql(action_query, con=engine)
df_actions["cntrb_id"] = df_actions["cntrb_id"].astype(str).str.slice(0, 20)
df_actions["created_at"] = pd.to_datetime(df_actions["created_at"], utc=True).dt.date

df_actions = df_actions.reset_index()
df_actions.drop("index", axis=1, inplace=True)

In [ ]:
df_actions

In [ ]:
df_actions["created_at"] = pd.to_datetime(df_actions["created_at"], utc=True)

# sort by created_at date latest to earliest and only keep a contributors most recent activity
df_actions = df_actions.sort_values(by="created_at", axis=0, ascending=False)
df_actions = df_actions.drop_duplicates(subset='cntrb_id', keep="first")

In [ ]:
# drop unneccessary columns not needed after preprocessing steps
df_actions = df_actions.reset_index()
df_actions.drop(["index","id","repo_name","login","action","rank" ], axis=1, inplace=True)

In [ ]:
df_actions

In [ ]:
# dictionary of cntrb_ids and their most recent activity on repo 
last_contrb = df_actions.set_index('cntrb_id')['created_at'].to_dict()

#### Get list of dates of the most recent activity for each contributor for each file 

In [ ]:
df_dynamic_directory["dates"] = df_dynamic_directory.apply(
        lambda row: [last_contrb[x] for x in row.cntrb_ids],
        axis=1,)

## Histogram testing

In [ ]:
plot_data = df_dynamic_directory

In [ ]:
plot_data = plot_data.explode('dates')

reformat into each row being a directory value and a date of one of the contributors most recent activity. Preprocessing step

In [ ]:
plot_data

Get files that have no contributors and remove from set to prevent errors in grouper function

In [ ]:
no_contribs = plot_data["directory_value"][plot_data.dates.isnull()].tolist()

In [ ]:
no_contribs

In [ ]:
plot_data = plot_data[~plot_data.dates.isnull()]

In [ ]:
plot_data

Creates df with a column for each month between start and end date. This will be used to confirm that there will be a column for every month even if there is no "last contribution" date in it. This greatly improves the heatmap ploting

In [ ]:
# dates based on action so it represents the length of the project
min_date = df_actions.created_at.min()
max_date = df_actions.created_at.max()
dates = pd.date_range(start=min_date, end=max_date, freq="M", inclusive="both")
df_fill = dates.to_frame(index=False, name="dates")

In [ ]:
final = pd.concat([plot_data, df_fill], axis=0)
final["directory_value"] = final["directory_value"].astype(str)

In [ ]:
final

Commenting out the groupby on the file_name level dataframe. This is grouping dates by every 2 months (another interval can be choosen) and counting the number of contributors with the last activity at that date

In [ ]:
final = final.groupby(pd.Grouper(key = 'dates', freq = '2M'))["directory_value"].value_counts().unstack(0)

In [ ]:
# removing the None row that was used for column formating
final.drop('nan', inplace = True)

In [ ]:
# add back the files that had no contributors
for files in no_contribs: 
    final.loc[files] = None 

In [ ]:
final

Plot of the heatmap

In [ ]:
fig = px.imshow(final, color_continuous_scale=px.colors.sequential.YlOrBr)

In [ ]:
#fig['layout']['xaxis']['autorange'] = "reversed"
fig['layout']['yaxis']['tickmode'] = "linear"
fig['layout']['height'] = 700
fig['layout']['coloraxis_colorbar_x'] = -0.15
fig['layout']['yaxis']['side'] = 'right'


In [ ]:
fig.show()

In [ ]:
fig.write_image("heatmap_directory.png")
Image(filename="heatmap_directory.png")

On the 8knot side I am thinking about 2 drop downs. One to select the repo and one for the directory. The repo one will be populated with the contents of the search bar (need to talk about how to get that) with the one chooose in base case is the first one in the list. The second drop down will be searchable and will be all of the directories/folders that are in the specific repository. Top level will be the default